In [ ]:
#packages import
import requests as rq
import numpy as np
import pandas as pd
import datetime as dt
import os
import re
import time

In [ ]:
#requests url and content
req = rq.get('https://www.118712.fr/annuaire/ville/roubaix-59/assurance-professionnelle')
page_content = req.content

In [ ]:
#BeautifulSoup
from bs4 import BeautifulSoup
bs_content = BeautifulSoup(page_content, 'html.parser')
bs_content

In [ ]:
#dynamic content -> Selenium
import selenium
from selenium import webdriver
#submodules and classes within selenium.webdriver.common
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
#manage exceptions tracking with common classes
from selenium.common.exceptions import NoSuchWindowException , TimeoutException, WebDriverException, NoSuchElementException

#set up chrome webdriver classes
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

#set up Webdriverwait and support classes
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [ ]:
#request
try:
    #Initialize Webdriver
    service = Service()
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    driver.get('https://www.118712.fr/recherche/auto/roubaix/assurance-professionnelle')

     

    #click PrivacyCompliance
    # button_Ok = driver.find_element(By.CSS_SELECTOR, '#privacy-cookie-banner__privacy-accept')
    # button_Ok.click()

    #find and count 'next page buttons' to define how many pages are to scrap
    buttons = driver.find_elements(By.CSS_SELECTOR, 'button.len1.adpJam')
    nbPage = 0

    #loop in buttons list to check if onclick attrbutes match with 'next buttons'
    for button in buttons:
        onclick_attribute = button.get_attribute('onclick')
        if onclick_attribute:
            pattern=r'changePageUseCurrentBounds\(\d+\)'
            match = re.search(pattern, onclick_attribute)
            # sum nb of pages
            if match:
                nbPage +=1
            else:
                continue
        else:
            continue
        
     #create empty lists in wich append informations as names(titles) and street address
    titles_all=[]
    addresses_all = []
    cities_all = []

    #loop through page results 

    for page in range(1,(nbPage+2)*2):

        # Wait for the elements to be visible
        elements_loaded = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'h2.h4')))
       # Fetch and print the text of the elements
        titles = [h2Elt.text   for h2Elt in elements_loaded]
        #print(titles)
        titles_all.append(titles)

        # Fetch and print the text of the address
        elements_addresses = driver.find_elements(By.CSS_SELECTOR, 'article > address.bi_adress.txt_md > p:first-child' )

        addresses = [address.text for address in elements_addresses]
        addresses_all.append(addresses)

        elements_cities = driver.find_elements(By.CSS_SELECTOR, 'article > address.bi_adress.txt_md > p:nth-child(2)' )
        cities = [citie.text for citie in elements_cities]
        cities_all.append(cities)
        
    #find phone numbers, displayed after clicking a first button
        #locate buttons 'fantomas' to click
        buttons_tobeclicked_located = driver.find_elements(By.CSS_SELECTOR, 'section.bi_list > article.bi.adp-map-item  > div.bi_cta > a.button.primary.bi_fantomas')
        
    #create an ampty list to add list numbers

        tel_nb_list = []
        for button in buttons_tobeclicked_located:
        
        # Move to the button before clicking with 'ActionChains' webdriver submodule ans classes
            actions = ActionChains(driver)
            actions.move_to_element(button).perform()
            #click button to display phone numbe
            button.click()
            # driver.implicitly_wait(2)
    
            #wait display and locate buttons mentionin tel number, dispayed after clicking initial button
            button_tel_located = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section.bi_list > article.bi.adp-map-item > div.bi_cta > a.button.primary.ab_test')))

            #collect list of "a" html elt      
            links = driver.find_elements(By.CSS_SELECTOR, 'a.button.primary.ab_test')

            #loop in buttons list to extract href attribute containing tel number
            for link in links: 
                tel_href = link.get_attribute('href')
                tel_nb_list.append(tel_href)

# click 'next button' by HTML id attribute ()
        next_button = driver.find_element(By.CSS_SELECTOR, 'button.len1.adpJam')
        next_button.click()

        # Optional: Add a delay to ensure the page fully loads before extracting results
        # time.sleep(2)

#manage alle possible exceptiosn
except NoSuchWindowException as NsWe:
    print(f"exception handled: {NsWe}")
except TimeoutException as te:
    print(f"TimeoutException: {te}")
    # Handle timeout exception
except WebDriverException as wde:
    print(f"WebDriverException: {wde}")
    # Handle WebDriver related exception
except NoSuchElementException as nse:
    print(f"NoSuchElementException: {nse}")
    # Handle NoSuchElementException
except Exception as ex:
    print(f"An unexpected exception occurred: {ex}")
    # Handle other exceptions

#close driver
finally:
    # Close the browser window in the finally block to ensure it's closed even if an exception occurs
    if 'driver' in locals() and driver is not None:
        driver.quit()


In [ ]:
# If you want a flat list containing titles from all pages, use extend
flat_titles_list = [title for titles_page in titles_all for title in titles_page]
print(flat_titles_list) 
flat_addresses_list = [address for addresses_page in addresses_all for address in addresses_page]
print(flat_addresses_list)  
flat_cities_list = [citie for cities_page in cities_all for citie in cities_page]
print(flat_cities_list)  
print(tel_nb_list)

In [ ]:
#define the number of rows and only use the half because iteratio on the double of page number + flip rows because seems to be better fitted on the second half
max_rows = (len(flat_titles_list)) 
flat_titles_list = np.array(flat_titles_list)[::-1]
flat_titles_list = flat_titles_list[:max_rows]
flat_addresses_list = np.array(flat_addresses_list)[::-1]
flat_addresses_list = flat_addresses_list[:max_rows]
flat_cities_list = np.array(flat_cities_list)[::-1]
flat_cities_list = flat_cities_list[:max_rows]
tel_nb_list = np.array(tel_nb_list)[::-1]
tel_nb_list = tel_nb_list[:max_rows]

In [ ]:
insurers_list2 = {'insurers': flat_titles_list,
                 'address' : flat_addresses_list,
                 'city': flat_cities_list,
                 'tel': tel_nb_list}
    
insurers_list2 = pd.DataFrame.from_dict(insurers_list2)
insurers_list2

In [ ]:
# insurers_list2[['zip_code', 'city']] = insurers_list2['city'].str.split(' ', n=1, expand=True)
# insurers_list2
# Split the 'city' column into 'zip code' and 'city' columns
# insurers_list2[['zip code', 'city']] = pd.DataFrame(insurers_list2['city'].assign(lambda x: x.str.split(' ', n=1, expand=True)))
insurers_list2[['zip code', 'city']] = insurers_list2['city'].str.split(n=1, expand=True)
# Display the updated DataFrame
print(insurers_list2)


In [ ]:
insurers_list2_cities_mask = ["Lille", "Lambersart", "Villeneuve D'Ascq", "Wasquehal", "Roubaix"]
mask = insurers_list2['city'].isin(insurers_list2_cities_mask )

In [ ]:
insurers_list2_f = insurers_list2.loc[mask]


In [ ]:
#export to csv
insurers_list2_f.to_csv('leads_insurers_RbxAndCo.csv', index=False)

SAND BOX


In [ ]:
import pandas as pd

# Assuming insurers_list2 is a dictionary-like object
# Convert it to a DataFrame
insurers_list2_df = pd.DataFrame.from_dict(insurers_list2)

# Group the DataFrame by the 'city' column and apply the all() function
insurers_list2_grouped = insurers_list2_df.groupby('city').tail()

# Print or use the resulting DataFrame
print(insurers_list2_grouped)


In [ ]:
import pandas as pd

# Assuming insurers_list2 is a DataFrame

# Create a boolean mask based on whether the 'city' column ends with 'Roubaix'
mask = insurers_list2['city'].str.endswith('baix')

# Filter the DataFrame using boolean indexing
filtered_insurers_list2 = insurers_list2[mask]

# Print or use the filtered DataFrame
print(filtered_insurers_list2)


In [ ]:
insurers_list2_noDup_f = insurers_list2.drop_duplicates(keep='last')
insurers_list2_noDup_f